# Apache Cassandra

本页面提供了一个使用 [Apache Cassandra®](https://cassandra.apache.org/) 作为向量存储的快速入门指南。

> [Cassandra](https://cassandra.apache.org/) 是一个 NoSQL、面向行的、高度可扩展且高度可用的数据库。从 5.0 版本开始，该数据库附带了 [向量搜索功能](https://cassandra.apache.org/doc/trunk/cassandra/vector-search/overview.html)。

_注意：除了访问数据库外，运行完整示例还需要 OpenAI API Key。_

### 设置和通用依赖项

集成使用需要以下 Python 包。

In [ ]:
%pip install --upgrade --quiet langchain-community "cassio>=0.1.4"

_注意：根据你的 LangChain 设置，你可能需要安装/升级此演示所需的其他依赖项_
_(特别是，需要最新版本的 `datasets`、`openai`、`pypdf` 和 `tiktoken`，以及 `langchain-community`)。_

In [ ]:
import os
from getpass import getpass

from datasets import (
    load_dataset,
)
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("OPENAI_API_KEY = ")

In [ ]:
embe = OpenAIEmbeddings()

## 导入向量存储

In [ ]:
from langchain_community.vectorstores import Cassandra

## 连接参数

本页所示的 Vector Store 集成可与 Cassandra 以及其他派生数据库（如 Astra DB）一起使用，这些数据库使用 CQL（Cassandra 查询语言）协议。

> DataStax [Astra DB](https://docs.datastax.com/en/astra-serverless/docs/vector-search/quickstart.html) 是一个基于 Cassandra 构建的托管无服务器数据库，提供相同的接口和优势。

根据您是通过 CQL 连接到 Cassandra 集群还是连接到 Astra DB，在创建 vector store 对象时将提供不同的参数。

### 连接 Cassandra 集群

您首先需要创建一个 `cassandra.cluster.Session` 对象，具体方法请参考[Cassandra 驱动程序文档](https://docs.datastax.com/en/developer/python-driver/latest/api/cassandra/cluster/#module-cassandra.cluster)。具体细节会有所不同（例如，涉及网络设置和身份验证），但这可能看起来像这样：

In [ ]:
from cassandra.cluster import Cluster

cluster = Cluster(["127.0.0.1"])
session = cluster.connect()

您现在可以将会话以及所需的 keyspace 名称设置为全局 CassIO 参数：

In [ ]:
import cassio

CASSANDRA_KEYSPACE = input("CASSANDRA_KEYSPACE = ")

cassio.init(session=session, keyspace=CASSANDRA_KEYSPACE)

现在，您可以创建向量存储：

In [ ]:
vstore = Cassandra(
    embedding=embe,
    table_name="cassandra_vector_demo",
    # session=None, keyspace=None  # Uncomment on older versions of LangChain
)

_注意：您也可以在创建 vector store 时将 session 和 keyspace 直接作为参数传递。然而，如果您的应用程序以多种方式使用 Cassandra（例如，用于向量存储、聊天记忆和 LLM 响应缓存），那么使用全局 `cassio.init` 设置会很方便，因为它允许在一个地方集中管理凭据和数据库连接。_

### 连接到 Astra DB

Astra DB 非常适合使用 CQL（Cassandra 查询语言）进行

在这种情况下，您可以使用以下连接参数初始化 CassIO：

- 数据库 ID，例如 `01234567-89ab-cdef-0123-456789abcdef`
- Token，例如 `AstraCS:6gBhNmsk135....` (必须是“数据库管理员” token)
- 可选的 Keyspace 名称（如果省略，将使用数据库的默认 Keyspace）

In [ ]:
ASTRA_DB_ID = input("ASTRA_DB_ID = ")
ASTRA_DB_APPLICATION_TOKEN = getpass("ASTRA_DB_APPLICATION_TOKEN = ")

desired_keyspace = input("ASTRA_DB_KEYSPACE (optional, can be left empty) = ")
if desired_keyspace:
    ASTRA_DB_KEYSPACE = desired_keyspace
else:
    ASTRA_DB_KEYSPACE = None

In [ ]:
import cassio

cassio.init(
    database_id=ASTRA_DB_ID,
    token=ASTRA_DB_APPLICATION_TOKEN,
    keyspace=ASTRA_DB_KEYSPACE,
)

现在您可以创建向量存储：

In [ ]:
vstore = Cassandra(
    embedding=embe,
    table_name="cassandra_vector_demo",
    # session=None, keyspace=None  # Uncomment on older versions of LangChain
)

## 加载数据集

将源数据集中的每一项转换为 `Document`，然后将它们写入向量存储：

In [ ]:
philo_dataset = load_dataset("datastax/philosopher-quotes")["train"]

docs = []
for entry in philo_dataset:
    metadata = {"author": entry["author"]}
    doc = Document(page_content=entry["quote"], metadata=metadata)
    docs.append(doc)

inserted_ids = vstore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents.")

在上面，`metadata` 字典是从源数据创建的，是 `Document` 的一部分。

添加更多条目，这次使用 `add_texts`：

In [ ]:
texts = ["I think, therefore I am.", "To the things themselves!"]
metadatas = [{"author": "descartes"}, {"author": "husserl"}]
ids = ["desc_01", "huss_xy"]

inserted_ids_2 = vstore.add_texts(texts=texts, metadatas=metadatas, ids=ids)
print(f"\nInserted {len(inserted_ids_2)} documents.")

_注意：您可能需要通过提高这些批量操作的并发级别来加速 `add_texts` 和 `add_documents` 的执行 - 请查看方法的 `batch_size` 参数以获取更多详细信息。具体哪些参数配置最优，可能会因网络和客户端机器的具体规格而有所不同。_

## 运行搜索

本节演示如何进行元数据过滤以及如何获取相似性分数：

In [ ]:
results = vstore.similarity_search("Our life is what we make of it", k=3)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

In [ ]:
results_filtered = vstore.similarity_search(
    "Our life is what we make of it",
    k=3,
    filter={"author": "plato"},
)
for res in results_filtered:
    print(f"* {res.page_content} [{res.metadata}]")

In [ ]:
results = vstore.similarity_search_with_score("Our life is what we make of it", k=3)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

### MMR（词项最大边际相关性）搜索

In [ ]:
results = vstore.max_marginal_relevance_search(
    "Our life is what we make of it",
    k=3,
    filter={"author": "aristotle"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

## 删除已存储的文档

In [ ]:
delete_1 = vstore.delete(inserted_ids[:3])
print(f"all_succeed={delete_1}")  # True, all documents deleted

In [ ]:
delete_2 = vstore.delete(inserted_ids[2:5])
print(f"some_succeeds={delete_2}")  # True, though some IDs were gone already

## 一个最小化的 RAG 链

接下来的单元格将实现一个简单的 RAG 流水线：
- 下载一个示例 PDF 文件并将其加载到 store 中；
- 使用 LCEL（LangChain Expression Language）创建一个 RAG 链，核心是 vector store；
- 运行问答链。

In [ ]:
!curl -L \
    "https://github.com/awesome-astra/datasets/blob/main/demo-resources/what-is-philosophy/what-is-philosophy.pdf?raw=true" \
    -o "what-is-philosophy.pdf"

In [ ]:
pdf_loader = PyPDFLoader("what-is-philosophy.pdf")
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64)
docs_from_pdf = pdf_loader.load_and_split(text_splitter=splitter)

print(f"Documents from PDF: {len(docs_from_pdf)}.")
inserted_ids_from_pdf = vstore.add_documents(docs_from_pdf)
print(f"Inserted {len(inserted_ids_from_pdf)} documents.")

In [ ]:
retriever = vstore.as_retriever(search_kwargs={"k": 3})

philo_template = """
You are a philosopher that draws inspiration from great thinkers of the past
to craft well-thought answers to user questions. Use the provided context as the basis
for your answers and do not make up new reasoning paths - just mix-and-match what you are given.
Your answers must be concise and to the point, and refrain from answering about other topics than philosophy.

CONTEXT:
{context}

QUESTION: {question}

YOUR ANSWER:"""

philo_prompt = ChatPromptTemplate.from_template(philo_template)

llm = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | philo_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke("How does Russel elaborate on Peirce's idea of the security blanket?")

更多内容，请通过 CQL 在此[这里](https://github.com/langchain-ai/langchain/tree/master/templates/cassandra-entomology-rag)查看使用 Astra DB 的完整 RAG 模板。

## 清理

这 essentially 使用 CassIO 中的 `Session` 对象，并用它运行一条 CQL `DROP TABLE` 语句：

_(您将丢失其中存储的数据。)_

In [ ]:
cassio.config.resolve_session().execute(
    f"DROP TABLE {cassio.config.resolve_keyspace()}.cassandra_vector_demo;"
)

### 了解更多

如需更多信息、扩展的快速入门指南和额外的用法示例，请访问 [CassIO 文档](https://cassio.org/frameworks/langchain/about/)，了解更多关于使用 LangChain `Cassandra` 向量存储的信息。

#### 归属声明

> Apache Cassandra、Cassandra 和 Apache 是 Apache 软件基金会（[Apache Software Foundation](http://www.apache.org/)）在美国和/或其他国家的注册商标或商标。